# Large Language Models, GPT3, LangChain and Chatbots

## Martin Callaghan
* Lecturer in CS, University of Leeds
* sometimes-RSE

Large language models (LLMs) are a transformative technology, enabling developers to build applications that they previously could not. But using these LLMs in isolation is often not enough to create a truly powerful app - the real power comes when you are able to combine them with other sources of computation or knowledge.

LangChain is a very new Python package that allows us to interact with LLMs and develop applications like Chatbots! This Notebook is based on some of the examples in the Langchain documentation.

See here for more information: https://langchain.readthedocs.io/en/latest/?

## Install some helper packages

You'll need to install Langchain (a Python package to build prompts and LLM toolchains, plus the OpenAI package to make using their API easier.

In [ ]:
# LangChain is the Python package that allws us to build prompts and language chains
!pip install langchain

In [ ]:
# We'll be using the OpenAI LLMs (including GPT3) so we need their API
!pip install openai

In [ ]:
# install the SERPAPI google results scraping package
!pip install google-search-results

## Register with Open AI

1. To use the Open AI API, you'll need to register with them. The API is **not** free but this Notebook shouldn't take more than 50p to run.

Register here: https://openai.com/blog/openai-api

2. You'll also need access to a Google search API via SERPAPI.

Register here: https://serpapi.com/


In [5]:
# We need to set up and use an API key to allow access to GPT-3 via an API
import os
os.environ["OPENAI_API_KEY"] = "<put-yout-API-key-here>"

In [6]:
# We also need access to a Google search API for a later exercise
os.environ["SERPAPI_API_KEY"] ="<put-yout-API-key-here>"

We will be using OpenAI's LLMs. There are many others we can use, such as the public (free) ones published by HuggingSpace or BLOOM.

The particular variant of GPT-3 provided through the default API is called InstructGPT.  

https://openai.com/blog/instruction-following/

It produces output that is better at following user intentions and is more truthful and less toxic.

Remember our principles of Responsible AI!

eg. from Microsoft: https://query.prod.cms.rt.microsoft.com/cms/api/am/binary/RE5cmFl


## 1. Getting some predictions from the LLM

The most basic building block of LangChain is calling an LLM on some input. Let’s look at a simple example of how to do this. 

For this example, let's say we are tryong to create an application that generates short and snappy names for our reserch projects.

In order to do this, we first need to import the LLM wrapper.

In [7]:
from langchain.llms import OpenAI

...and then set the "temperature" of the results. Temperature is an option passed into the LLM. A temperature of 0 means the response is deterministic, in that it always returns the same result. A temperature of greater than zero results in increasing randomness in the answer.  

Here we set temperature to be 0.9 to give a random output (for OpenAI's LLMs the temperature should be between 0 and 1).

You can read more about temperature [here](https://algowriting.medium.com/gpt-3-temperature-setting-101-41200ff0d0be).

In [8]:
# Generate some random outputs
llm = OpenAI (temperature = 0.9)

In [9]:
text = "What would be a good name for a research project that creates chatbots for student education"

In [ ]:
print (llm(text))

Try running the cell above a few times and you'll get different answers.

## 2. Managing prompts for LLMs

Normally when you use an LLM in an application, you are not sending user input directly to the LLM. Instead, you are probably taking user input and constructing a prompt, and then sending that to the LLM.

In [12]:
# Define a template for a prompt

from langchain.prompts import PromptTemplate

prompt = PromptTemplate (
    input_variables= ["project_name"],
    template = "What would be a good name for a research project that investigates {project_name}?",
)

In [ ]:
print (prompt.format (project_name = "Chatbot-Enhanced Student Education"))

## 3. Combining LLMs and prompts into Chains

A real application is not just one primitive (such as a PromptTemplate or a simple prediction), but rather a combination of them.

A chain in LangChain is made up of links, which can be either primitives like LLMs or other chains.

The most core type of chain is an LLMChain, which consists of a PromptTemplate and an LLM.

In [14]:
# Extending the previous example, we can construct an LLMChain which takes user input, 
# formats it with a PromptTemplate, and then passes the formatted response to an LLM.

from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["project_name"],
    template="What would be a good name for a research project that investigates {project_name}?",
)

We can now create a very simple chain that will take user input, format the prompt with it, and then send it to the LLM:

In [15]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
chain.run ("Chatbot-Enhanced Student Education")

## 4. Calling chains dynamically based on user input

So far the chains we’ve looked at run in a predetermined order.

Agents no longer do: they use an LLM to determine which actions to take and in what order. An action can either be using a tool and observing its output, or returning to the user.

Some concepts:

* **Tool**: A function that performs a specific duty. This can be things like: Google Search, Database lookup, Python REPL, other chains. The interface for a tool is currently a function that is expected to have a string as an input, with a string as an output.
* **LLM**: The language model powering the agent.
* **Agent**: The agent to use. This should be a string that references a support agent class. Because this notebook focuses on the simplest, highest level API, this only covers using the standard supported agents. 

**Agents**: https://langchain.readthedocs.io/en/latest/modules/agents/agents.html

**Tools**: https://langchain.readthedocs.io/en/latest/modules/agents/tools.html

In [17]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI

In [18]:
# First, let's load the language model we're going to use to control the agent.
llm = OpenAI(temperature=0)

In [19]:
# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)

In [20]:
# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [ ]:
# Ask it a question
agent.run("Who is Brooklyn Beckham's wife? What is her current age raised to the 0.23 power?")

In [ ]:
# Let's try another one:
agent.run(" Who was the lead singer of 'The Animals'? How did he die?")

You  might want to see if this is true!

https://en.wikipedia.org/wiki/Eric_Burdon

How do you think this happened?  What is the reasoning here?

## 5. Adding state or memory to our chains

Often, you may want a chain or agent to have some concept of “memory” so that it may remember information about its previous interactions. The clearest and simple example of this is when designing a chatbot - you want it to remember previous messages so it can use context from that to have a better conversation. This would be a type of “short-term memory”.

LangChain provides several specially created chains just for this purpose. This notebook walks through using one of those chains (the ConversationChain) with two different types of memory.

In [ ]:
from langchain import OpenAI, ConversationChain

llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

conversation.predict(input="Hi there!")

In [ ]:
conversation.predict(input="I'm doing well! Just having a conversation with an AI.")

# Replicating ChatGPT (sort of)

A simple chain that replicates ChatGPT by combining (1) a specific prompt, and (2) the concept of memory.

In [25]:
from langchain import OpenAI, ConversationChain, LLMChain, PromptTemplate
from langchain.chains.conversation.memory import ConversationalBufferWindowMemory


template = """Assistant is a large language model trained by OpenAI.
Assistant is designed to be able to assist with a wide range of tasks, from 
answering simple questions to providing in-depth explanations and discussions 
on a wide range of topics. 

As a language model, Assistant is able to generate human-like text based on the 
input it receives, allowing it to engage in natural-sounding conversations and 
provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are 
constantly evolving. It is able to process and understand large amounts of text, 
and can use this knowledge to provide accurate and informative responses to a 
wide range of questions. Additionally, Assistant is able to generate its own 
text based on the input it receives, allowing it to engage in discussions and 
provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks 
and provide valuable insights and information on a wide range of topics. 
Whether you need help with a specific question or just want to have a 
conversation about a particular topic, Assistant is here to assist.

{history}
Human: {human_input}
Assistant:"""

prompt = PromptTemplate(
    input_variables=["history", "human_input"], 
    template=template
)


chatgpt_chain = LLMChain(
    llm=OpenAI(temperature=0), 
    prompt=prompt, 
    verbose=True, 
    memory=ConversationalBufferWindowMemory(k=2),
)

In [ ]:
output = chatgpt_chain.predict(human_input= """I want you to act as a Linux 
terminal. I will type commands and you will reply with what the terminal should 
show. I want you to only reply wiht the terminal output inside one unique code 
block, and nothing else. Do not write explanations. Do not type commands unless 
I instruct you to do so. When I need to tell you something in English I will 
do so by putting text inside curly brackets {like this}. My first command is pwd.""")
print(output)

In [ ]:
output = chatgpt_chain.predict(human_input="ls ~")
print(output)

In [ ]:
output = chatgpt_chain.predict(human_input="cd ~")
print(output)

In [ ]:
output = chatgpt_chain.predict(human_input="{Please make a file jokes.txt inside and put some jokes inside}")
print(output)

In [ ]:
output = chatgpt_chain.predict(human_input="""echo -e "x=lambda y:y*5+3;print('Result:' + str(x(6)))" > run.py && python3 run.py""")
print(output)

## This example opens up a whole new area of work called **Prompt Engineering**
(basically how to ask an LLM questions)

There's a good Medium article all about that [here](https://medium.com/mlearning-ai/from-zero-shot-to-chain-of-thought-prompt-engineering-choosing-the-right-prompt-types-88800f242137 )



# More applications using LangChain

https://langchain.readthedocs.io/en/latest/gallery.html